In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#ref:https://www.kaggle.com/sinaasappel/tutorial-multiple-regression

In [9]:
## Importing packages
library(tidyverse) # metapackage with lots of helpful functions
library(gridExtra) # to plot multiple ggplots aside
library(car) # package for regression diagnostics

## Reading in files
ElecticityAnalytics <- read.csv("../input/electicity-usage-combined-analytics/Electicity_Usage_Combined_Analytics.csv")

In [10]:
## have a peak at the data
head(ElecticityAnalytics)
str(ElecticityAnalytics)
summary(ElecticityAnalytics)

,electricitytype,sector,codelist_province,year,population_n,area_km2,ratio_population_per_area,electicity_user_amount,house_usage,smallbusiness_usage,⋯,mean_minimum_temperature,mean_msl_pressure,mean_relative_humidity,mean_station_pressure,std_dry_bulb_temperature,std_maximum_temperature,std_minimum_temperature,std_msl_pressure,std_relative_humidity,std_station_pressure
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,3,50,2549,1779254,20107,88,495959,628112378,279831691,⋯,19.02,1008.7,72.57,938.92,2.26,2.31,2.67,3.3,12.59,1.52
2,1,3,50,2550,1779254,20107,88,514462,681441804,306045872,⋯,19.02,1008.7,72.57,938.92,2.26,2.31,2.67,3.3,12.59,1.52
3,1,3,50,2551,1779254,20107,88,528748,709318633,319069606,⋯,19.02,1008.7,72.57,938.92,2.26,2.31,2.67,3.3,12.59,1.52
4,1,3,50,2552,1779254,20107,88,548591,756632234,336265580,⋯,19.02,1008.7,72.57,938.92,2.26,2.31,2.67,3.3,12.59,1.52
5,1,3,50,2553,1779254,20107,88,567000,843631167,363137055,⋯,19.02,1008.7,72.57,938.92,2.26,2.31,2.67,3.3,12.59,1.52
6,1,3,50,2554,1779254,20107,88,585408,834128201,366614246,⋯,19.02,1008.7,72.57,938.92,2.26,2.31,2.67,3.3,12.59,1.52


'data.frame':	630 obs. of  41 variables:
 $ electricitytype            : num  1 1 1 1 1 1 1 1 1 1 ...
 $ sector                     : num  3 3 3 3 3 3 3 3 3 3 ...
 $ codelist_province          : num  50 50 50 50 50 50 50 50 50 57 ...
 $ year                       : num  2549 2550 2551 2552 2553 ...
 $ population_n               : num  1779254 1779254 1779254 1779254 1779254 ...
 $ area_km2                   : num  20107 20107 20107 20107 20107 ...
 $ ratio_population_per_area  : num  88 88 88 88 88 88 88 88 88 111 ...
 $ electicity_user_amount     : num  495959 514462 528748 548591 567000 ...
 $ house_usage                : num  6.28e+08 6.81e+08 7.09e+08 7.57e+08 8.44e+08 ...
 $ smallbusiness_usage        : num  2.80e+08 3.06e+08 3.19e+08 3.36e+08 3.63e+08 ...
 $ mediumbusiness_usage       : num  3.11e+08 3.45e+08 3.52e+08 3.78e+08 4.06e+08 ...
 $ largebusiness_usage        : num  2.51e+08 2.79e+08 3.01e+08 3.05e+08 3.24e+08 ...
 $ largebusiness_usage_code   : num  1 1 1 1 1 1 1 1 1 1

 electricitytype      sector      codelist_province      year     
 Min.   :0.0000   Min.   :0.000   Min.   :10.00     Min.   :2549  
 1st Qu.:1.0000   1st Qu.:2.000   1st Qu.:27.00     1st Qu.:2551  
 Median :1.0000   Median :2.000   Median :47.50     Median :2553  
 Mean   :0.9571   Mean   :2.471   Mean   :50.59     Mean   :2553  
 3rd Qu.:1.0000   3rd Qu.:4.000   3rd Qu.:74.00     3rd Qu.:2555  
 Max.   :1.0000   Max.   :4.000   Max.   :96.00     Max.   :2557  
                                                                  
  population_n        area_km2     ratio_population_per_area
 Min.   : 193305   Min.   :  417   Min.   :  22.0           
 1st Qu.: 478227   1st Qu.: 3424   1st Qu.:  85.0           
 Median : 655766   Median : 5436   Median : 126.0           
 Mean   : 867977   Mean   : 6684   Mean   : 254.1           
 3rd Qu.:1068010   3rd Qu.: 9598   3rd Qu.: 172.0           
 Max.   :5666264   Max.   :20494   Max.   :3612.0           
                                     

In [ ]:
plot.age <- ggplot(ElecticityAnalytics, aes(x = total_usage, y = population_n)) +
 geom_point()

plot.bmi <- ggplot(ElecticityAnalytics, aes(x = total_usage, y = population_n)) +
 geom_point()

grid.arrange(plot.age, plot.bmi, ncol=2)

In [ ]:
# Fit model
library(MASS)
fit <- lm(total_usage ~ electricitytype + sector + population_n + area_km2 + #ratio_population_per_area + 
            electicity_user_amount + largebusiness_usage_code + all_factory_amount + all_registered_capital + all_worker_amount + 
            average_monthly_income + average_monthly_expenditure + average_debt_per_household + number_of_household + 
            mean_dry_bulb_temperature + mean_maximum_temperature + mean_minimum_temperature + mean_msl_pressure + 
            mean_relative_humidity + mean_station_pressure + std_dry_bulb_temperature + std_maximum_temperature + 
            std_minimum_temperature + std_msl_pressure + std_relative_humidity + std_station_pressure, 
          data = ElecticityAnalytics)
# Show the model
print(fit)
summary(fit)

In [11]:
#Stepwise multiple regression -- total_usage
library(MASS)
lmMod1 <- lm(total_usage ~ population_n + area_km2 + electicity_user_amount + largebusiness_usage_code + 
              all_factory_amount + all_registered_capital + all_worker_amount + average_monthly_income + average_monthly_expenditure + 
              average_debt_per_household + mean_dry_bulb_temperature + mean_maximum_temperature + 
              mean_minimum_temperature + mean_msl_pressure + mean_relative_humidity + mean_station_pressure,
            data = ElecticityAnalytics)
selectedMod1 <- step(lmMod1)
summary(selectedMod1)
coefficients(selectedMod1)


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select




Start:  AIC=26042.23
total_usage ~ population_n + area_km2 + electicity_user_amount + 
    largebusiness_usage_code + all_factory_amount + all_registered_capital + 
    all_worker_amount + average_monthly_income + average_monthly_expenditure + 
    average_debt_per_household + mean_dry_bulb_temperature + 
    mean_maximum_temperature + mean_minimum_temperature + mean_msl_pressure + 
    mean_relative_humidity + mean_station_pressure

                              Df  Sum of Sq        RSS   AIC
- mean_dry_bulb_temperature    1 2.3969e+17 5.3515e+20 26040
- mean_minimum_temperature     1 2.7522e+17 5.3518e+20 26041
<none>                                      5.3491e+20 26042
- mean_relative_humidity       1 1.7381e+18 5.3664e+20 26042
- average_monthly_expenditure  1 2.0539e+18 5.3696e+20 26043
- population_n                 1 2.5036e+18 5.3741e+20 26043
- largebusiness_usage_code     1 2.9241e+18 5.3783e+20 26044
- mean_maximum_temperature     1 3.3142e+18 5.3822e+20 26044
- average_mon


Call:
lm(formula = total_usage ~ population_n + area_km2 + electicity_user_amount + 
    largebusiness_usage_code + all_factory_amount + all_registered_capital + 
    all_worker_amount + average_monthly_income + average_monthly_expenditure + 
    average_debt_per_household + mean_maximum_temperature + mean_msl_pressure + 
    mean_relative_humidity + mean_station_pressure, data = ElecticityAnalytics)

Residuals:
       Min         1Q     Median         3Q        Max 
-3.076e+09 -6.384e+08  8.768e+07  5.759e+08  3.972e+09 

Coefficients:
                              Estimate Std. Error t value Pr(>|t|)    
(Intercept)                  5.921e+11  1.098e+11   5.392 9.94e-08 ***
population_n                -8.629e+01  5.240e+01  -1.647 0.100158    
area_km2                    -3.897e+04  9.671e+03  -4.029 6.30e-05 ***
electicity_user_amount       1.247e+04  4.086e+02  30.527  < 2e-16 ***
largebusiness_usage_code    -6.573e+08  3.608e+08  -1.822 0.068927 .  
all_factory_amount          -4

(Intercept)                population_n 
               5.921366e+11               -8.628591e+01 
                   area_km2      electicity_user_amount 
              -3.896732e+04                1.247400e+04 
   largebusiness_usage_code          all_factory_amount 
              -6.573192e+08               -4.625209e+05 
     all_registered_capital           all_worker_amount 
               3.741405e+03                1.671784e+04 
     average_monthly_income average_monthly_expenditure 
               3.517374e+04                2.453461e+04 
 average_debt_per_household    mean_maximum_temperature 
              -3.880271e+03               -1.758731e+08 
          mean_msl_pressure      mean_relative_humidity 
              -6.394890e+08               -3.309071e+07 
      mean_station_pressure 
               6.035881e+07

In [ ]:
#Outliers and influential cases
# obtain residuals, cooks distance for selectedMod and add to dataframe
ElecticityAnalytics$standardized.residuals <- rstandard(selectedMod1)
ElecticityAnalytics$cooks.distance <- cooks.distance(selectedMod1) # to spot influential cases

# indicate which standardized residuals are > 2 and add to dataframe
ElecticityAnalytics$large.residual2 <- ElecticityAnalytics$standardized.residuals > 2 | ElecticityAnalytics$standardized.residuals < -2

# indicate which standardized residuals are > 2.5 and add to dataframe
ElecticityAnalytics$large.residual2.5 <- ElecticityAnalytics$standardized.residuals > 2.5 | ElecticityAnalytics$standardized.residuals < -2.5

# indicate which cases are outliers (standardized residual > 3)
ElecticityAnalytics$outlier.residual <- ElecticityAnalytics$standardized.residuals > 3 | ElecticityAnalytics$standardized.residuals < -3

# percentage of cases with residual > 2 (this should 5% or lower)
paste("The percentage of cases with a standardized residual > 2 is", 
      round(sum(ElecticityAnalytics$large.residual2)/nrow(ElecticityAnalytics)*100, 2), "percent.")

# percentage of cases with residual > 2.5 (this should be 1% or lower)
paste("The percentage of cases with a standardized residual > 2.5 is", 
      round(sum(ElecticityAnalytics$large.residual2.5)/nrow(ElecticityAnalytics)*100, 2), "percent.")

# potential outliers
nrow(ElecticityAnalytics[which(ElecticityAnalytics$outlier.residual==T),])

In [ ]:
#Checking the assumption of indepence
dwt(selectedMod1)

In [ ]:
#Checking the assumption of multicollinearity
# VIF
vif(selectedMod1)
# tolerance
1/vif(selectedMod1)
# mean VIF
mean(vif(selectedMod1))

In [ ]:
#Checking assumptions about residuals
plot(selectedMod1)

In [12]:
#install.packages("zoo")
library(lmtest)
bptest(selectedMod1)


	studentized Breusch-Pagan test

data:  selectedMod1
BP = 263.52, df = 14, p-value < 2.2e-16


In [13]:
ncvTest(selectedMod1)

Non-constant Variance Score Test 
Variance formula: ~ fitted.values 
Chisquare = 343.9176, Df = 1, p = < 2.22e-16

In [ ]:
#Stepwise multiple regression -- house_usage
library(MASS)
lmMod2 <- lm(house_usage ~ electricitytype + sector + population_n + area_km2 + electicity_user_amount + #largebusiness_usage_code + 
              all_factory_amount + all_registered_capital + all_worker_amount + average_monthly_income + average_monthly_expenditure + 
              average_debt_per_household + number_of_household + mean_dry_bulb_temperature + mean_maximum_temperature + 
              mean_minimum_temperature + mean_msl_pressure + mean_relative_humidity + mean_station_pressure + std_dry_bulb_temperature + 
              std_maximum_temperature + std_minimum_temperature + std_msl_pressure + std_relative_humidity + std_station_pressure, 
            data = ElecticityAnalytics)
selectedMod2 <- step(lmMod2)
summary(selectedMod2)
coefficients(selectedMod2)

In [ ]:
#Stepwise multiple regression -- smallbusiness_usage
library(MASS)
lmMod3 <- lm(smallbusiness_usage ~ electricitytype + sector + population_n + area_km2 + electicity_user_amount + #largebusiness_usage_code + 
              all_factory_amount + all_registered_capital + all_worker_amount + average_monthly_income + average_monthly_expenditure + 
              average_debt_per_household + number_of_household + mean_dry_bulb_temperature + mean_maximum_temperature + 
              mean_minimum_temperature + mean_msl_pressure + mean_relative_humidity + mean_station_pressure + std_dry_bulb_temperature + 
              std_maximum_temperature + std_minimum_temperature + std_msl_pressure + std_relative_humidity + std_station_pressure, 
            data = ElecticityAnalytics)
selectedMod3 <- step(lmMod3)
summary(selectedMod3)
coefficients(selectedMod3)

In [ ]:
#Stepwise multiple regression -- largebusiness_usage
library(MASS)
lmMod4 <- lm(largebusiness_usage ~ electricitytype + sector + population_n + area_km2 + electicity_user_amount + largebusiness_usage_code + 
              all_factory_amount + all_registered_capital + all_worker_amount + average_monthly_income + average_monthly_expenditure + 
              average_debt_per_household + number_of_household + mean_dry_bulb_temperature + mean_maximum_temperature + 
              mean_minimum_temperature + mean_msl_pressure + mean_relative_humidity + mean_station_pressure + std_dry_bulb_temperature + 
              std_maximum_temperature + std_minimum_temperature + std_msl_pressure + std_relative_humidity + std_station_pressure, 
            data = ElecticityAnalytics)
selectedMod4 <- step(lmMod4)
summary(selectedMod4)
coefficients(selectedMod4)

In [ ]:
#Stepwise multiple regression -- other_usage
library(MASS)
lmMod5 <- lm(other_usage ~ electricitytype + sector + population_n + area_km2 + electicity_user_amount + largebusiness_usage_code + 
              all_factory_amount + all_registered_capital + all_worker_amount + average_monthly_income + average_monthly_expenditure + 
              average_debt_per_household + number_of_household + mean_dry_bulb_temperature + mean_maximum_temperature + 
              mean_minimum_temperature + mean_msl_pressure + mean_relative_humidity + mean_station_pressure + std_dry_bulb_temperature + 
              std_maximum_temperature + std_minimum_temperature + std_msl_pressure + std_relative_humidity + std_station_pressure, 
            data = ElecticityAnalytics)
selectedMod5 <- step(lmMod5)
summary(selectedMod5)
coefficients(selectedMod5)